In [2]:
# Import packages
import pandas as pd
import numpy as np
import os

root = os.getcwd()

In [ ]:
# Importing datasets from HDRUK Winter Pressures project
# TODO: You will need to replace the filename with your actual apc winter pressures filename
acute = pd.read_csv(f"{root}/data/apc_winter_pressures_file.csv")

In [3]:
# Importing ICD-10 code mappings used by our trust - enter your excel/csv for mapping in the quotations
icd = pd.read_excel(f"{root}/icd_10_mapper.xlsx")

In [4]:
# Changing the names of the columns we would like from code list and creating a diagnosis column
icd2 = icd[icd['CODING_METHOD']==10]
icd_codes = icd2[['DIAGNOSIS_CODE', 'DESCRIPTION']]
icd_codes.columns = ['diag_01', 'diagnosis']

In [ ]:
# Creating new dataframe merged to include a diagnosis field based on ICD-10 codes. 
acute_new = pd.merge(acute, icd_codes, on='diag_01', how='left')

In [ ]:
# truncating dataframe to essential variables
acute2 = acute_new[['diagnosis',
                    'length_of_stay',
                    'admiage',
                    'townsend_score_quintile',
                    'gender_cat',
                    'ethnos_cat',
                    'in_hours',
                    'disdest_cat',
                    'opertn_count',
                    'comorb_count']].copy()

In [ ]:
# Creating dataframe of the highest ranking diagnoses in order
diag = pd.DataFrame(acute2['diagnosis'].value_counts()).reset_index()

In [ ]:
# Creating a list of the top 50 ranked diagnoses by frequency
top_fifty = diag.iloc[0:50, 0].to_list()

In [ ]:
# Creating new dataframe of just those with the 50 most common diagnoses 
# works out at around half of the original cohort in UHS dataset
acute_top = acute2[acute2['diagnosis'].isin(top_fifty)]

In [ ]:
# Setting range for low number suppression which will be part of pipeline and used below
lownum = range(1, 8)

In [ ]:
# Creating tables for overall descriptive stats using top 50 diagnoses

overall_catvar = acute_top[['diagnosis', 'gender_cat', 'ethnos_cat', 'in_hours', 'disdest_cat']] # slicing dataframe into just categorical variables
overall_contvar = acute_top[['admiage', 'townsend_score_quintile', 'length_of_stay', 'opertn_count', 'comorb_count']] # slicing dataframe into just continuous variables

overall_catvar_for_concat = {} # empty dictionary to populate with dataframes ready for concatination into tables at the end

for col in overall_catvar: # for loop to populate dictionary with dataframes of counts for each column
            overall_catvar_for_concat[col] = pd.DataFrame(overall_catvar[col].value_counts(dropna=False))
            overall_catvar_for_concat[col].columns = ['count']
            overall_catvar_for_concat[col] = overall_catvar_for_concat[col].replace(lownum, -1)

overall_cat_table = pd.concat(overall_catvar_for_concat) # once dictionary populated, these dataframes are concatinated
overall_cat_table['percentage'] = (overall_cat_table['count'] / len(acute_top) * 100).round(1) # percentage column added

overall_cont_table = pd.DataFrame(overall_contvar.describe().T) # continuous variables described
overall_cont_table.columns = ['count', 'mean', 'std', 'min', '25%', 'median', '75%', 'max']
overall_cont_table['IQR'] = overall_cont_table['75%'] - overall_cont_table['25%'] # creating interquartile range column
overall_cont_table['n_missing'] = len(acute_top) - overall_cont_table['count'] # creating missing number column with percentage
overall_cont_table['percent_missing'] = overall_cont_table['n_missing'] / len(acute_top) * 100
overall_cont_table = overall_cont_table.round(1)
overall_cont_table['count'] = overall_cont_table['count'].replace(lownum, -1)

In [ ]:
# Creating for loop that populates dictionaries with a table for each value in the top 50 diagnoses list. 
# Once these are made, they're concatinated into big dataframes with every diagnosis' descriptive statistics
# PLEASE ADJUST TOWNSEND SCORE QUINTILE INTO CATEGORICAL VARIABLES IF DESIRED

cat_for_concat = {} # empty dictionaries to populate
cont_for_concat = {}

for val in top_fifty:
    df = acute_top[acute_top['diagnosis']== val] # slicing new dataframe of only the listed diagnosis
    df = df.drop(columns='diagnosis') # dropping the diagnosis column otherwise pointless variable

    cat_df = df[['gender_cat', 'ethnos_cat', 'in_hours', 'disdest_cat']] # choosing which variables are designated as categorical or continuous
    cont_df = df[['admiage', 'townsend_score_quintile', 'length_of_stay', 'opertn_count', 'comorb_count']]

    catvar_for_concat = {} # creating empty data dictionary to populate with small dataframes

    for col in cat_df: # for loop to populate dictionary
            catvar_for_concat[col] = pd.DataFrame(cat_df[col].value_counts(dropna=False).sort_index(ascending=True))
            catvar_for_concat[col].columns = ['count']
            catvar_for_concat[col] = catvar_for_concat[col].replace(lownum, -1)
    cat_table = pd.concat(catvar_for_concat) # once dictionary populated, these dataframes are concatinated
    cat_table['percentage'] = (cat_table['count'] / len(df) * 100).round(1) # percentage column added

    cont_table = pd.DataFrame(cont_df.describe().T) # continuous variables described
    cont_table.columns = ['count', 'mean', 'std', 'min', '25%', 'median', '75%', 'max']
    cont_table['IQR'] = cont_table['75%'] - cont_table['25%'] # creating interquartile range column
    cont_table['n_missing'] = len(df) - cont_table['count'] # creating missing number column with percentage
    cont_table['percent_missing'] = cont_table['n_missing'] / len(df) * 100
    cont_table = cont_table.round(1)
    cat_for_concat[f'{val}'] = cat_table # populating dictionary of dataframes with a table for each diagnosis in the top fifty list
    cont_for_concat[f'{val}'] = cont_table
    
categorical_table = pd.concat(cat_for_concat) # concatinating all the tables into one big one for each
continuous_table = pd.concat(cont_for_concat)
continuous_table['count'] = continuous_table['count'].replace(lownum, -1)

In [ ]:
# Output the results

with pd.ExcelWriter(f'{root}\hdruk_outcome_output.xlsx') as writer:
    continuous_table.to_excel(writer, sheet_name='Continuous per diagnosis')
    overall_cat_table.to_excel(writer, sheet_name='Overall categorical')
    categorical_table.to_excel(writer, sheet_name='Categorical per diagnosis')
    overall_cont_table.to_excel(writer, sheet_name='Overall continuous')